In [17]:
import requests
from copy import copy


def dfs(category_id):
    url = "https://api.itemscout.io/api/category/{0}/subcategories".format(category_id)
    response = requests.get(url = url)
    
    for d in response.json()['data']:
        d_id = d['id']
        d_is_leaf = d['is_leaf']
        
        if(d_is_leaf):
            leaf = copy(d_id)
            leaf_category.append(leaf)
            
        else:
            dfs(d_id)
    
paths = []
leaf_category = []
starting_category = [1,2,3,4,5,8,9] #출산/육아(6) , 식품 제외(7)

dfs(1)

#for i in starting_category:
#    dfs(i)


print(leaf_category)

[660, 683, 684, 685, 686, 687, 688, 689, 690, 691, 692, 693, 694, 695, 696, 697, 698, 699, 700, 701, 702, 703, 704, 705, 706, 707, 708, 709, 3540, 3541, 3542, 3543, 3544, 3545, 3546, 3547, 3548, 3549, 3550, 3551, 3552, 3706, 710, 711, 712, 713, 714, 715, 716, 717, 718, 719, 720, 721, 722, 723, 724, 1948, 45823, 725, 726, 727, 728, 3701, 3702, 3703, 3704, 3705]


In [18]:
cookies = {
    'NNB': 'VFILOFFQY5YGI',
    'SHP_BUCKET_ID': '2',
    'spage_uid': '',
}

headers = {
    'authority': 'search.shopping.naver.com',
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    # 'cookie': 'NNB=VFILOFFQY5YGI; SHP_BUCKET_ID=2; spage_uid=',
    'logic': 'PART',
    'referer': 'https://search.shopping.naver.com/search/all?frm=NVSHMDL&origQuery=%EB%8B%AD%ED%84%B8%EB%BD%91%EB%8A%94%EA%B8%B0%EA%B3%84&pagingIndex=1&pagingSize=40&productSet=model&query=%EB%8B%AD%ED%84%B8%EB%BD%91%EB%8A%94%EA%B8%B0%EA%B3%84&sort=rel&timestamp=&viewType=list',
    'sbth': '15b5a3d3cfd6c114f2e2c67dc134288b5bfb45b3cdfa3405bf1a36bc3ac566191c871d1f00d1a5073220541c0cce4387',
    'sec-ch-ua': '"Google Chrome";v="113", "Chromium";v="113", "Not-A.Brand";v="24"',
    'sec-ch-ua-arch': '"x86"',
    'sec-ch-ua-bitness': '"64"',
    'sec-ch-ua-full-version-list': '"Google Chrome";v="113.0.5672.127", "Chromium";v="113.0.5672.127", "Not-A.Brand";v="24.0.0.0"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-model': '""',
    'sec-ch-ua-platform': '"Windows"',
    'sec-ch-ua-platform-version': '"10.0.0"',
    'sec-ch-ua-wow64': '?0',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36',
}

params = {
    'eq': '',
    'frm': 'NVSHTTL',
    'iq': '',
    'origQuery': '닭털뽑는기계',
    'pagingIndex': '1',
    'pagingSize': '40',
    'productSet': 'total',
    'query': '닭털뽑는기계',
    'sort': 'rel',
    'viewType': 'list',
    'window': '',
    'xq': '',
}


In [22]:
import json

def get_keywords(data):
    res_keywords=[]
    
    for d in data:
        res_keywords.append(d['keyword'])
    
    return res_keywords

#브랜드 의심 키워드 제거
def remove_brands(keywords, brands):
    res_keywords = []
    
    for keyword in keywords:
        is_brand = 0
        
        for brand in brands:
            if brand in keyword:
                is_brand=1
                break
                
        if is_brand==0:
            res_keywords.append(keyword)
    
    return res_keywords

#한글자 브랜드 제거
def remove_a_word (brands):
    res_brands = []
    
    for brand in brands:
        if len(brand)>1:
            res_brands.append(brand)
    
    return res_brands

payload={
    'duration':'30d',
    'genders':'f,m',
    'age':'10,60'
}

final_keywords = []

for c in leaf_category[:1]:
    url_data = "https://api.itemscout.io/api/category/{0}/data".format(c)
    url_brands = "https://api.itemscout.io/api/category/{0}/brands".format(c)
    response_data = requests.post(url_data, json=payload)
    response_brands = requests.get(url_brands)
    data = response_data.json()['data']['data'].values()
    brands = response_brands.json()['data']
    brands = remove_a_word(brands)
    
    keywords = get_keywords(data)
    keywords_nb = remove_brands(keywords,brands)
    
    for keyword in keywords_nb[:20]:
        
        params['origQuery'] = keyword
        params['query'] = keyword

        response = requests.get('https://search.shopping.naver.com/api/search/all', params=params, cookies=cookies, headers=headers)

        '''['productSetFilter', 'searchAdResult', 'mallShortCut', 'queryValidateResult', 'shoppingResult', 'appliedSmartPriceSort', 'searchBanInfo', 'eventBanner', 'searchParam', 'partialSearched', 'surls', 'nClick'])'''
        cnt_oversea = 0
        
        for product in response.json()['shoppingResult']['products']:
            if product['overseaTp'] == '1':
                cnt_oversea+=1
        
        #count = text.count("해외")
        print(keyword,"해외배송 개수 :", cnt_oversea)
        

니트투피스 해외배송 개수 : 0
블라우스 해외배송 개수 : 0
원피스 해외배송 개수 : 0
나시가디건세트 해외배송 개수 : 0
여름가디건 해외배송 개수 : 0
여름니트 해외배송 개수 : 0
투피스 해외배송 개수 : 1
여름자켓 해외배송 개수 : 0
린넨셔츠 해외배송 개수 : 7
린넨나시 해외배송 개수 : 0
시밀러룩 해외배송 개수 : 0
커플룩 해외배송 개수 : 8
아노락 해외배송 개수 : 1
홀복 해외배송 개수 : 10
니트투피스세트 해외배송 개수 : 2
니트세트 해외배송 개수 : 0
세트 해외배송 개수 : 0
등하원룩 해외배송 개수 : 6
린넨자켓세트 해외배송 개수 : 9
트레이닝세트 해외배송 개수 : 2


In [16]:
import requests
import json
from copy import copy

cookies = {
    'NNB': 'VFILOFFQY5YGI',
    'SHP_BUCKET_ID': '2',
    'spage_uid': '',
}

headers = {
    'authority': 'search.shopping.naver.com',
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    # 'cookie': 'NNB=VFILOFFQY5YGI; SHP_BUCKET_ID=2; spage_uid=',
    'logic': 'PART',
    'referer': 'https://search.shopping.naver.com/search/all?frm=NVSHMDL&origQuery=%EB%8B%AD%ED%84%B8%EB%BD%91%EB%8A%94%EA%B8%B0%EA%B3%84&pagingIndex=1&pagingSize=40&productSet=model&query=%EB%8B%AD%ED%84%B8%EB%BD%91%EB%8A%94%EA%B8%B0%EA%B3%84&sort=rel&timestamp=&viewType=list',
    'sbth': '15b5a3d3cfd6c114f2e2c67dc134288b5bfb45b3cdfa3405bf1a36bc3ac566191c871d1f00d1a5073220541c0cce4387',
    'sec-ch-ua': '"Google Chrome";v="113", "Chromium";v="113", "Not-A.Brand";v="24"',
    'sec-ch-ua-arch': '"x86"',
    'sec-ch-ua-bitness': '"64"',
    'sec-ch-ua-full-version-list': '"Google Chrome";v="113.0.5672.127", "Chromium";v="113.0.5672.127", "Not-A.Brand";v="24.0.0.0"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-model': '""',
    'sec-ch-ua-platform': '"Windows"',
    'sec-ch-ua-platform-version': '"10.0.0"',
    'sec-ch-ua-wow64': '?0',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36',
}

params = {
    'eq': '',
    'frm': 'NVSHTTL',
    'iq': '',
    'origQuery': '닭털뽑는기계',
    'pagingIndex': '1',
    'pagingSize': '40',
    'productSet': 'total',
    'query': '닭털뽑는기계',
    'sort': 'rel',
    'viewType': 'list',
    'window': '',
    'xq': '',
}

params['origQuery'] = '장화'
params['query'] = '장화'

response = requests.get('https://search.shopping.naver.com/api/search/all', params=params, cookies=cookies, headers=headers)

'''['productSetFilter', 'searchAdResult', 'mallShortCut', 'queryValidateResult', 'shoppingResult', 'appliedSmartPriceSort', 'searchBanInfo', 'eventBanner', 'searchParam', 'partialSearched', 'surls', 'nClick'])'''

for product in response.json()['shoppingResult']['products']:
    print(product['overseaTp'])
#print(response.json()['shoppingResult']['products'][0]['overseaTp'])

0
1
0
0
0
0
0
0
0
0
1
1
0
1
1
0
0
0
0
0
0
0
0
0
1
0
1
0
0
0
0
0
1
1
1
0
0
0
0
0
